# 协程
- http://segmentfault.com/a/1190000009781688
- 实现协程比较好的包有asyncio, tornado, gevent
- 定义：从技术角度讲，协程就是一个你可以暂停执行的函数，或者干脆把协程理解成生成器
- 实现：
    - yield返回
    - send调用
- 协程的4个状态
    - inspect.getgeneratorstate(...) 函数确定，该函数会返回下述字符串中的一个：
        - GEN_CTREATED: 等待开始执行
        - GEN_RUNNING: 解释器正在执行
        - GEN_SUSPENED: 在yield表达式处暂停
        - GEN_CLOSED: 执行结束
    - next预激（prime）
- 协程终止
    - 协程中未处理的异常会向上冒泡，传给next函数或send方法的调用方（即触发协程的对象）
    - 终止协程的一种方式：发送某个哨符值，让协程退出。内置的None和Ellipse等常量经常用作哨符值
    
- yield from
    - 调用协程为了得到返回值，协程必须正常终止
    - 生成器正常终止会发出StopIteration异常，异常对象的value属性保存返回值
    - yield from 从内部捕获StopIteration异常
    - yield from后面加上可迭代对象，他可以把可迭代对象里的每个元素一个一个的yield出来
    - 案例03
    - 概念：
        1. 调用方：调用委派生成器的客户端（调用方）代码
        2. 委托生成器：包含yield from表达式的生成器函数
        3. 子生成器：yield from后面加的生成器函数
    - 委派生成器
        - 包含yield from表达式的生成器函数
        - 委派生成器在yield from表达式处暂停，调用方可以直接把数据发给子生成器
        - 子生成器再把产出的值发给调用方
        - 子生成器在最后，解释器会抛出StopIteration，并把返回值附加到异常对象上
        - 作用：在调用方与子生成器之间建立一个双向通道
        - 案例04

In [3]:
# 协程代码案例01
def simple_coroutine():
    print('-> start')
    x = yield
    print('-> received', x)

# 创建生成器对象
sc = simple_coroutine()
print(1111)

# 可以使用sc.send(None), 效果一样
# 调用
next(sc) # 预激

print(2222)
sc.send('zhexiao') # x 接收这个数据

# 执行到最后了，没有了，就会爆StopIteration异常，这才是正确的 

1111
-> start
2222
-> received zhexiao


StopIteration: 

In [7]:
# 案例02
def simple_coroutine(a):
    print('-> start')
    
    b = yield a  # 返回 a=5
    print('-> received', a, b)
    
    c = yield a + b
    print('-> received', a, b, c)

# 创建生成器对象
sc = simple_coroutine(5)

# 调用（预激）
aa = next(sc)  # aa接收返回值
print(aa)

bb = sc.send(6) # 函数里的b接收，b=6; bb接收返回的值
print(bb)

cc = sc.send(7) # 函数里的c接收，c=7
# print(cc)

-> start
5
-> received 5 6
11
-> received 5 6 7


StopIteration: 

In [11]:
# 案例03：yield form

def gen():
    for i in 'AB':
        yield i
print(list(gen()))

def new_gen():
    yield from 'AB'
    
print(list(new_gen()))

['A', 'B']
<generator object new_gen at 0x00000251EAA76E08>


In [14]:
# 案例04：委派生成器

# 子生成器：yield from后面加的生成器函数
def average_gen():
    total = 0
    count = 0
    average = 0
    while True:
        new_num = yield average # 返回average, 然后又继续调用，接着上次的地方执行，所以一直在循环
        if new_num is None:
            break
        count += 1
        total += new_num
        average = total/count

    # 每一次return，都意味着当前协程结束。
    return total,count,average

# 委托生成器：包含yield from表达式的函数
def proxy_gen():
    while True:
        # 只有子生成器要结束（return）了，yield from左边的变量才会被赋值，后面的代码才会执行。
        total, count, average = yield from average_gen()
        print("计算完毕！！\n总共传入 {} 个数值， 总和：{}，平均数：{}".format(count, total, average))

# 调用方
def main():
    calc_average = proxy_gen()
    next(calc_average)            # 预激协程
    print(calc_average.send(10))  # 打印：10.0
    print(calc_average.send(20))  # 打印：15.0
    print(calc_average.send(30))  # 打印：20.0
    calc_average.send(None)      # 结束协程
    # 如果此处再调用calc_average.send(10)，由于上一协程已经结束，将重开一协程

if __name__ == '__main__':
    main()

10.0
15.0
20.0
计算完毕！！
总共传入 3 个数值， 总和：60，平均数：20.0


# asyncio
- https://www.liaoxuefeng.com/wiki/1016959663602400/1017970488768640
- 本身是一个消息循环，内置对异步io的支持
- 步骤：
    - 创建消息循环
    - 把协程导入
    - 关闭

In [ ]:
import asyncio

# 使用了装饰器@asyncio.coroutine来将这个get_html()函数定义为协程
@asyncio.coroutine
def get_html(url, name):
    print("%s get %s html start" % (name, url))
    yield from asyncio.sleep(2)
    print("%s get %s html end" % (name, url))


if __name__ == '__main__':
    # 启动消息（事件）循环
    loop = asyncio.get_event_loop()
    # 创建两个协程
    tasks = [
        get_html("http://www.baidu.com", "A"),
        get_html("http://www.souhu.com", "B"),
    ]
    # 启动事件循环并将协程放进去执行
    loop.run_until_complete(asyncio.wait(tasks))
    # 关闭消息循环
    loop.close()